In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from xgboost import plot_importance
import sklearn
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split


%matplotlib inline

In [4]:
df3 = pd.read_csv('../../sf16_ds4/local/data_mcnulty/loan_app_data.csv')

In [8]:
df3.head()

,loan_amnt,title,dti,zip_code,addr_state,emp_length,Accepted
0,14000.0,Medical expenses,15.97,439xx,OH,4 years,1
1,25000.0,Credit card refinancing,9.54,054xx,VT,1 year,1
2,16000.0,Credit card refinancing,33.18,297xx,SC,10+ years,1
3,15000.0,Debt consolidation,13.89,973xx,OR,3 years,1
4,15000.0,Credit card refinancing,13.24,085xx,NJ,6 years,1


In [21]:
df_nozip = df3.drop(['zip_code'],axis=1)

In [23]:
df_nozip.head()

,loan_amnt,title,dti,addr_state,emp_length,Accepted
0,14000.0,Medical expenses,15.97,OH,4 years,1
1,25000.0,Credit card refinancing,9.54,VT,1 year,1
2,16000.0,Credit card refinancing,33.18,SC,10+ years,1
3,15000.0,Debt consolidation,13.89,OR,3 years,1
4,15000.0,Credit card refinancing,13.24,NJ,6 years,1


In [25]:
X = df_nozip.drop(['Accepted'],axis=1)
y = df_nozip['Accepted']

In [30]:
X = pd.get_dummies(X)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3280130 entries, 0 to 3280129
Data columns (total 77 columns):
loan_amnt                        float64
dti                              float64
title_Auto                       float64
title_Business                   float64
title_Credit card refinancing    float64
title_Debt consolidation         float64
title_Educational                float64
title_Green loan                 float64
title_Home buying                float64
title_Home improvement           float64
title_Major purchase             float64
title_Medical expenses           float64
title_Moving and relocation      float64
title_Other                      float64
title_Renewable energy           float64
title_Vacation                   float64
addr_state_AK                    float64
addr_state_AL                    float64
addr_state_AR                    float64
addr_state_AZ                    float64
addr_state_CA                    float64
addr_state_CO             

In [32]:
X_parse = X.drop(['emp_length_< 1 year'],axis=1)
X_parse

,loan_amnt,dti,title_Auto,title_Business,title_Credit card refinancing,title_Debt consolidation,title_Educational,title_Green loan,title_Home buying,title_Home improvement,...,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_n/a
0,14000.0,15.97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,25000.0,9.54,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,16000.0,33.18,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15000.0,13.89,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15000.0,13.24,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,24000.0,10.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,2500.0,17.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,28800.0,18.01,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,10000.0,10.81,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,35000.0,6.46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
X_train, X_test, y_train, y_test = train_test_split(X_parse,y,test_size=0.3,random_state=0)

In [47]:
gbm = xgb.XGBClassifier(max_depth=2, n_estimators=200, learning_rate=0.1).fit(X_train, y_train)
predictions = gbm.predict(X_test)
print gbm.score(X_test, y_test)

0.936788074456
